In [1]:
import pandas as pd

In [7]:
X_test = pd.read_csv('../data/test_features.csv', index_col=0)

In [3]:
predictions = pd.read_csv('cat_5k_inv_perm.csv', index_col=0)

In [8]:
pseudo_y = predictions[(predictions.radiant_win_prob > .9) | (predictions.radiant_win_prob < .1)]

In [14]:
pseudo_targets = pseudo_y.radiant_win_prob.map(lambda x: 0.0 if x < .5 else 1.0)

In [17]:
pseudo_X = X_test.loc[pseudo_targets.index, :]

In [18]:
X = pd.read_csv('../data/train_features.csv', index_col=0)
Y = pd.read_csv('../data/train_targets.csv', index_col=0)

In [30]:
X_enh = pd.concat([X, pseudo_X])
Y_enh = pd.concat([Y.radiant_win, pseudo_targets])

In [31]:
Y_enh = pd.DataFrame(Y_enh, columns=['radiant_win'])

In [26]:
X_enh.shape, Y_enh.shape

((41492, 245), (41492, 1))

In [23]:
X_enh.to_csv('../data/pseudo_train_features.csv')

/Volumes/Storage/goinfre/ptyshevs/env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [32]:
Y_enh.to_csv('../data/pseudo_train_targets.csv')

In [46]:
def pseudo_labels(predictions, threshold=.1):
    pseudo_y = predictions[(predictions.radiant_win_prob > (1 - threshold)) | (predictions.radiant_win_prob < threshold)]
    
    return pseudo_y.radiant_win_prob.map(lambda x: False if x < .5 else True)
    
def pseudo_enhance(X, Y, X_test, predictions, threshold=.1):
    pseudo_targets = pseudo_labels(predictions, threshold)
    
    pseudo_X = X_test.loc[pseudo_targets.index, :]
    
    return pd.concat([X, pseudo_X]), pd.DataFrame(pd.concat([Y.radiant_win, pseudo_targets]), columns=['radiant_win'])

In [47]:
pseudo_labels(predictions)

match_id_hash
70e5ba30f367cea48793b9003fab9d38     True
4d9ef74d3a2025d79e9423105fd73d41     True
b891f03bb2a86d78b84043437fc95e04     True
ff898afdb5bb5c7163bde45a009503f7    False
f4e59a3443f4d1e55b8639651dca3a4d    False
                                    ...  
977a1db0b5295f6f463abc20d16b36ec    False
29355f338e4ee84c68dcbdcb3858374c    False
6bd9779a71650348aa3cd75f45278bfb     True
9376a283b50779433de829c79529fe2c    False
1bbaf6d71e197ebf96c6afa3c921000a    False
Name: radiant_win_prob, Length: 1817, dtype: bool

In [42]:
X_try, Y_try = pseudo_labels(X, Y, X_test, predictions)